## Import Dependencies

In [20]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import datetime as dt
import json
import geojson
from math import cos, asin, sqrt
from config import username, password

## Extract CSV File

### Store CSV Into Dataframe

In [2]:
#Store COBRA crime data CSV as dataframe
cobra_csv_file = "../../data/COBRA-2022.csv"
cobra_complete_df = pd.read_csv(cobra_csv_file)
cobra_complete_df.head()

,offense_id,rpt_date,occur_date,occur_day,occur_day_num,occur_time,poss_date,poss_time,beat,zone,location,ibr_code,UC2_Literal,neighborhood,npu,lat,long
0,22011609,7/20/2022,7/20/2022,Wednesday,4.0,18:30,7/20/2022,19:00,607,6,"1395 CUSTER WAY SE\nATLANTA, GA 30316\nUNITED ...",23H,LARCENY-NON VEHICLE,Custer/McDonough/Guice,W,33.716073,-84.353217
1,22254073,9/11/2022,9/11/2022,Sunday,1.0,11:15,9/11/2022,11:45,606,6,"777 MEMORIAL DR SE\nATLANTA, GA 30316\nUNITED ...",23F,LARCENY-FROM VEHICLE,Reynoldstown,N,33.746335,-84.361753
2,202800283,6/14/2022,10/6/2020,Tuesday,3.0,05:21,10/6/2020,05:40,413,4,"4050 BLANTON AVE SW\nATLANTA, GA 30331\nUNITED...",23F,LARCENY-FROM VEHICLE,Fairburn,P,33.690551,-84.520970
3,203140048,2/21/2022,11/9/2020,Monday,2.0,00:32,11/9/2020,01:49,409,4,"2111 CAMPBELLTON RD SW\nATLANTA, GA 30311\nUNI...",13A,AGG ASSAULT,Adams Park,R,33.706526,-84.458000
4,203250206,4/15/2022,11/20/2020,Friday,6.0,01:40,11/20/2020,01:45,109,1,"2621 DONALD LEE HOLLOWELL PKWY NW\nATLANTA, GA...",23H,LARCENY-NON VEHICLE,Center Hill,J,33.776168,-84.472729


## Transform COBRA crime dataframe

In [3]:
#Create a new dataframe with just the columns we want to use
cobra_summary_df = cobra_complete_df[["offense_id", "occur_date", "UC2_Literal", "neighborhood", "lat", "long"]]
cobra_summary_df.head()

,offense_id,occur_date,UC2_Literal,neighborhood,lat,long
0,22011609,7/20/2022,LARCENY-NON VEHICLE,Custer/McDonough/Guice,33.716073,-84.353217
1,22254073,9/11/2022,LARCENY-FROM VEHICLE,Reynoldstown,33.746335,-84.361753
2,202800283,10/6/2020,LARCENY-FROM VEHICLE,Fairburn,33.690551,-84.520970
3,203140048,11/9/2020,AGG ASSAULT,Adams Park,33.706526,-84.458000
4,203250206,11/20/2020,LARCENY-NON VEHICLE,Center Hill,33.776168,-84.472729


In [4]:
#Rename column for easier readability
cobra_summary_df.rename(columns={'UC2_Literal':'crime_type'}, inplace = True)
cobra_summary_df.head()

C:\AnacondaPython\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,offense_id,occur_date,crime_type,neighborhood,lat,long
0,22011609,7/20/2022,LARCENY-NON VEHICLE,Custer/McDonough/Guice,33.716073,-84.353217
1,22254073,9/11/2022,LARCENY-FROM VEHICLE,Reynoldstown,33.746335,-84.361753
2,202800283,10/6/2020,LARCENY-FROM VEHICLE,Fairburn,33.690551,-84.520970
3,203140048,11/9/2020,AGG ASSAULT,Adams Park,33.706526,-84.458000
4,203250206,11/20/2020,LARCENY-NON VEHICLE,Center Hill,33.776168,-84.472729


In [5]:
#Convert 'occur_date' column from a string type to a date type
cobra_summary_df["occur_date"] = pd.to_datetime(cobra_summary_df["occur_date"])

C:\AnacondaPython\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
#Filter out any rows with an 'occur_date' that is NOT in 2022
cobra_summary_df = cobra_summary_df.loc[cobra_summary_df["occur_date"] > "2021-12-31"]
cobra_summary_df.head()

,offense_id,occur_date,crime_type,neighborhood,lat,long
0,22011609,2022-07-20,LARCENY-NON VEHICLE,Custer/McDonough/Guice,33.716073,-84.353217
1,22254073,2022-09-11,LARCENY-FROM VEHICLE,Reynoldstown,33.746335,-84.361753
13,210770926,2022-01-15,LARCENY-NON VEHICLE,Arlington Estates,33.688746,-84.544190
21,212630747,2022-07-11,HOMICIDE,Fairburn Heights,33.772107,-84.501745
60,220010108,2022-01-01,AGG ASSAULT,Summerhill,33.738249,-84.383948


In [7]:
#Get rid of any rows with a duplicate 'offense_id'
cobra_summary_df = cobra_summary_df.drop_duplicates(subset='offense_id', keep="first")

# reset index 
cobra_summary_df.reset_index(drop=True)

,offense_id,occur_date,crime_type,neighborhood,lat,long
0,22011609,2022-07-20,LARCENY-NON VEHICLE,Custer/McDonough/Guice,33.716073,-84.353217
1,22254073,2022-09-11,LARCENY-FROM VEHICLE,Reynoldstown,33.746335,-84.361753
2,210770926,2022-01-15,LARCENY-NON VEHICLE,Arlington Estates,33.688746,-84.544190
3,212630747,2022-07-11,HOMICIDE,Fairburn Heights,33.772107,-84.501745
4,220010108,2022-01-01,AGG ASSAULT,Summerhill,33.738249,-84.383948
...,...,...,...,...,...,...
21007,223560731,2022-12-22,LARCENY-NON VEHICLE,Edgewood,33.757886,-84.347406
21008,223560734,2022-12-19,AUTO THEFT,NaN,33.753515,-84.495354
21009,223560793,2022-12-12,LARCENY-NON VEHICLE,Downtown,33.748887,-84.392788
21010,223560803,2022-12-22,LARCENY-NON VEHICLE,Harland Terrace,33.751137,-84.486120


## Read in MARTA rail station file

In [ ]:
#Store MARTA rail station data CSV as dataframe
rail_station_csv_file = "../../data/Transit_Rail_Stations.csv"
rail_station_complete_df = pd.read_csv(rail_station_csv_file)
rail_station_complete_df.head()

In [ ]:
#Create a new dataframe with just the columns we want to use
rail_station_summary_df = rail_station_complete_df[["STATION", "latitude", "longitude"]]
rail_station_summary_df.head()

In [ ]:
#Rename column for easier readability
rail_station_summary_df.rename(columns={'STATION':'station'}, inplace = True)
rail_station_summary_df.head()

## Find distance in lat and long of crime from rail station

In [ ]:
#Put the rail station dataframe into a list of dictionaries
rail_station_data = rail_station_summary_df.to_dict(orient='records')
rail_station_data

In [ ]:
#Put Crime dataframe into a list of dictionaries
cobra_summary_data = cobra_summary_df.to_dict(orient='records')

In [ ]:
#Functions for finding the closest MARTA rail station to each crime
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav))

def closest(data, v):
    return min(data, key=lambda p: distance(v['latitude'],v['longitude'],p['latitude'],p['longitude']))
                                                   
v = {'latitude': 33.716073, 'longitude': -84.353217}
print(closest(rail_station_data, v))

In [ ]:
#Create empty lists to hold closest rail station data
closest_station_name = []
closest_station_lat = []
closest_station_long = []
distance_from_station = []

In [ ]:
#Loop through each crime and find the closest MARTA rail station
for i in range(len(cobra_summary_data)):
    crime_lat = cobra_summary_data[i]['lat']
    crime_lon = cobra_summary_data[i]['long']
    v = {'latitude': crime_lat, 'longitude': crime_lon}
    closest_rail_station = closest(rail_station_data, v)
    closest_station_name.append(closest_rail_station['STATION'])
    closest_station_lat.append(closest_rail_station['latitude'])
    closest_station_long.append(closest_rail_station['longitude'])
    distance_to_station = distance(crime_lat, crime_lon, closest_rail_station['latitude'], closest_rail_station['longitude'])
    distance_from_station.append(distance_to_station)


In [ ]:
#Create empty lists to hold distance data
diff_in_lat = []
diff_in_long = []

In [ ]:
#Loop through each crime and find the difference in latitude and longitude from closest MARTA rail station
for j in range(len(cobra_summary_data)):
    lat_difference = abs(closest_station_lat[j] - cobra_summary_data[j]['lat'])
    diff_in_lat.append(lat_difference)
    long_difference = abs(closest_station_long[j] - cobra_summary_data[j]['long'])
    diff_in_long.append(long_difference)

In [ ]:
#Append new columns to cobra_summary_df dataframe
cobra_summary_df['closest_station'] = closest_station_name
cobra_summary_df['difference_in_lat'] = diff_in_lat
cobra_summary_df['difference_in_long'] = diff_in_long
cobra_summary_df['distance_away'] = distance_from_station

cobra_summary_df.head()

## Export tables to Postgres database

In [ ]:
#Connect to database
protocol = 'postgresql'
host = 'localhost'
port = 5432
database_name = 'secret_of_nimby'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
#Check for tables
engine.table_names()

In [ ]:
#Load in the cobra_summary table
cobra_summary_df.to_sql(name='cobra_summary', con=engine, if_exists='append', index=False)

In [ ]:
#Load in the transit rail station table
rail_station_summary_df.to_sql(name='transit_rail_station', con=engine, if_exists='append', index=False)

## Read in Atlanta Neighborhood CSV

In [24]:
#Store Atlanta Neighborhood data CSV as dataframe
neighborhood_data_csv_file = "../../data/Atlanta_Neighborhood_Data_raw.csv"
neighborhood_data_complete_df = pd.read_csv(neighborhood_data_csv_file)
neighborhood_data_complete_df.head()

,GEOID,Details,Median age (years) 2020,% Non-Hispanic White 2020,% Non-Hispanic Black 2020,% Hispanic all races 2020,Median household income 2020,# Total population 2020,% Non-Hispanic other race adults 2020,% Non-Hispanic Asian or Pacific Islander 2020
0,A01,"Margaret Mitchell, Paces, Pleasant Hill",46.674712,73.7,13.7,4.6,NaN,4720,2.4,4.2
1,A02,"Kingswood, Mt. Paran/Northside, Mt. Paran Park...",46.560749,81.9,3.8,4.2,131744.630,4489,2.6,5.8
2,A03,"Chastain Park, Tuxedo Park",38.233601,88.3,1.3,2.9,188532.320,3578,1.9,3.6
3,B01,Peachtree Heights West,54.781736,67.9,18.7,5.4,83711.796,5799,3.3,4.0
4,B02,"Buckhead Forest, South Tuxedo Park",33.576254,68.2,12.9,8.8,108796.380,4893,3.8,5.3


In [25]:
#Only keep the columns we need an rearrange
neighborhood_data_summary_df = neighborhood_data_complete_df[["GEOID", "Details", "# Total population 2020", "Median age (years) 2020", "Median household income 2020", "% Non-Hispanic White 2020", "% Non-Hispanic Black 2020", "% Hispanic all races 2020", "% Non-Hispanic Asian or Pacific Islander 2020", "% Non-Hispanic other race adults 2020"]]
neighborhood_data_summary_df.head()

,GEOID,Details,# Total population 2020,Median age (years) 2020,Median household income 2020,% Non-Hispanic White 2020,% Non-Hispanic Black 2020,% Hispanic all races 2020,% Non-Hispanic Asian or Pacific Islander 2020,% Non-Hispanic other race adults 2020
0,A01,"Margaret Mitchell, Paces, Pleasant Hill",4720,46.674712,NaN,73.7,13.7,4.6,4.2,2.4
1,A02,"Kingswood, Mt. Paran/Northside, Mt. Paran Park...",4489,46.560749,131744.630,81.9,3.8,4.2,5.8,2.6
2,A03,"Chastain Park, Tuxedo Park",3578,38.233601,188532.320,88.3,1.3,2.9,3.6,1.9
3,B01,Peachtree Heights West,5799,54.781736,83711.796,67.9,18.7,5.4,4.0,3.3
4,B02,"Buckhead Forest, South Tuxedo Park",4893,33.576254,108796.380,68.2,12.9,8.8,5.3,3.8


In [26]:
#Rename columns for easier readability
neighborhood_data_summary_df.rename(columns={'GEOID':'geoid','Details':'neighborhood','# Total population 2020': 'total_population', 'Median age (years) 2020':'median_age', 'Median household income 2020':'median_household_income', '% Non-Hispanic White 2020':'percent_white', '% Non-Hispanic Black 2020':'percent_black', '% Hispanic all races 2020':'percent_hispanic', '% Non-Hispanic Asian or Pacific Islander 2020':'percent_asian_or_pacific_islander', '% Non-Hispanic other race adults 2020':'percent_other_races'}, inplace = True)
neighborhood_data_summary_df.head()

,geoid,neighborhood,total_population,median_age,median_household_income,percent_white,percent_black,percent_hispanic,percent_asian_or_pacific_islander,percent_other_races
0,A01,"Margaret Mitchell, Paces, Pleasant Hill",4720,46.674712,NaN,73.7,13.7,4.6,4.2,2.4
1,A02,"Kingswood, Mt. Paran/Northside, Mt. Paran Park...",4489,46.560749,131744.630,81.9,3.8,4.2,5.8,2.6
2,A03,"Chastain Park, Tuxedo Park",3578,38.233601,188532.320,88.3,1.3,2.9,3.6,1.9
3,B01,Peachtree Heights West,5799,54.781736,83711.796,67.9,18.7,5.4,4.0,3.3
4,B02,"Buckhead Forest, South Tuxedo Park",4893,33.576254,108796.380,68.2,12.9,8.8,5.3,3.8


## Export neighborhood data table to postgres

In [27]:
#Connect to database
protocol = 'postgresql'
host = 'localhost'
port = 5432
database_name = 'secret_of_nimby'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [28]:
#Check for tables
engine.table_names()

['cobra_complete',
 'cobra_summary',
 'transit_rail_station',
 'neighborhood_data']

In [29]:
#Load in the neighborhood data table
neighborhood_data_summary_df.to_sql(name='neighborhood_data', con=engine, if_exists='append', index=False)